# 📘 1. Introducción
En este notebook aprenderemos a:
- Leer datos **estructurados** (CSV, Parquet).
- Leer datos **semiestructurados** (JSON, anidado).
- Manejar **esquemas explícitos** para optimización.
- Usar **funciones de PySpark** para transformar datos.
- Aplicar técnicas de **optimización en Databricks**: cache, repartition, broadcast, Delta Lake.

📂 2. Importación de librerías y configuración inicial

In [0]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark.sql.functions as F

# 📊 3. Lectura de datos estructurados (CSV público)

In [0]:
# URL del archivo
url = "https://raw.githubusercontent.com/owid/covid-19-data/refs/heads/master/public/data/owid-covid-data.csv"
# ⚠️ Nota: no se puede leer directamente con spark.read.csv(url) debido a restricciones de permisos en Databricks.
# Por eso lo cargamos primero con Pandas y luego lo convertimos a Spark DataFrame.

# Leer con pandas
pdf = pd.read_csv(url)

# Convertir a Spark DataFrame
df_tmp = spark.createDataFrame(pdf)

# Definir esquema manualmente (solo las columnas que queremos)
schema = StructType([
    StructField("iso_code", StringType(), True),
    StructField("continent", StringType(), True),
    StructField("location", StringType(), True),
    StructField("date", StringType(), True),          # puede cambiarse a DateType si se requiere
    StructField("total_cases", DoubleType(), True),
    StructField("new_cases", DoubleType(), True),
    StructField("total_deaths", DoubleType(), True),
    StructField("population", DoubleType(), True)
])

# Seleccionar y castear columnas al esquema definido usando F.col
df_covid = (
    df_tmp
    .select(
        F.col("iso_code").cast(StringType()),
        F.col("continent").cast(StringType()),
        F.col("location").cast(StringType()),
        F.col("date").cast(StringType()),     # o DateType si prefieres
        F.col("total_cases").cast(DoubleType()),
        F.col("new_cases").cast(DoubleType()),
        F.col("total_deaths").cast(DoubleType()),
        F.col("population").cast(DoubleType())
    )
)

# Mostrar datos
df_covid.show(5)
df_covid.printSchema()

# 🛠 4. Transformaciones en datos estructurados

In [0]:
df_covid = df_covid.withColumn("date", F.to_date("date", "yyyy-MM-dd"))
df_covid = df_covid.withColumn("cases_per_million", (F.col("total_cases") / F.col("population")) * 1e6)
df_colombia = df_covid.filter(F.col("location") == "Colombia")
display(df_colombia)

# 📂 5. Lectura de datos semiestructurados (JSON público)

In [0]:
import requests
import json
import os

# URL del API
json_url = "https://api.open-meteo.com/v1/forecast?latitude=6.2442&longitude=-75.5812&hourly=temperature_2m"
# ⚠️ No se puede leer directamente con spark.read.json(json_url) por restricciones en Databricks.
# Por eso descargamos primero con requests y guardamos local.

# Descargar el JSON
response = requests.get(json_url)
data = response.json()

# Guardar en un archivo temporal
local_path = "/tmp/weather.json"
with open(local_path, "w") as f:
    json.dump(data, f)

# Cargar el JSON con Spark
df_weather = spark.read.json("file:" + local_path)

df_weather.printSchema()
df_weather.show(2, truncate=False)

# 🔍 6. Manejo de datos anidados en JSON

In [0]:
df_weather_expanded = df_weather.select(
    "latitude",
    "longitude",
    "hourly.time",
    "hourly.temperature_2m"
)
df_weather_expanded.show(5, truncate=False)

# ⚡ 7. Optimización en Databricks / PySpark

En el procesamiento distribuido con Spark, la **optimización** es fundamental para mejorar el rendimiento y reducir costos de cómputo.  
Aquí explicamos tres técnicas clave:

### 🗂 1. `cache()` / `persist()`
- Cuando ejecutamos transformaciones en un DataFrame, Spark no guarda los resultados inmediatamente (evaluación diferida).  
- Si el mismo DataFrame se usa varias veces, Spark lo recalcularía cada vez.  
- Usar `cache()` o `persist()` permite **almacenar en memoria** los resultados intermedios, evitando recomputaciones.  
- 🚀 Beneficio: acelera consultas repetidas a costa de usar más memoria.

### 2. repartition() y coalesce()

- Spark divide los datos en particiones, que son las unidades que se distribuyen entre los nodos del clúster.
- repartition(n, col) → redistribuye los datos en n particiones de forma balanceada (puede implicar un shuffle costoso).
- coalesce(n) → reduce el número de particiones sin shuffle completo, útil cuando queremos consolidar archivos de salida.
- 🚀 Beneficio: controlar el número de particiones evita problemas como subutilización (muy pocas particiones) o sobrecarga (demasiadas particiones pequeñas).

### 📡 3. broadcast()

- En un join, si una de las tablas es pequeña, se puede replicar en todos los nodos en vez de hacer un shuffle completo.
- broadcast(df) le dice a Spark que use esa estrategia.
- 🚀 Beneficio: reduce drásticamente el tiempo y el costo de joins cuando trabajamos con tablas de referencia pequeñas.

In [0]:
df_covid.cache()
df_covid.count()

df_covid = df_covid.repartition(10, "location")

from pyspark.sql.functions import broadcast
df_pop = df_covid.select("location", "population").distinct()
df_joined = df_covid.join(broadcast(df_pop), "location")

df_covid.write.format("delta").mode("overwrite").save("/mnt/delta/covid")
df_delta = spark.read.format("delta").load("/mnt/delta/covid")

# 📈 8. Consultas analíticas (Window Functions)

Las **Window Functions** en Spark permiten hacer cálculos avanzados sobre un conjunto de filas **relacionadas** con la fila actual, sin necesidad de agrupar toda la tabla.  

📌 En otras palabras: nos permiten calcular métricas como acumulados, rankings o valores anteriores/siguientes, mientras seguimos viendo todas las filas originales.

### ⚙️ ¿Cómo funcionan?
- Se definen con un **Window Specification** que indica:
  - **PARTITION BY** → cómo dividir los datos en grupos (ej: por país).
  - **ORDER BY** → cómo ordenar dentro de cada grupo (ej: por fecha).

### 🛠 Tipos comunes de funciones de ventana:

1. Cálculo con filas anteriores o siguientes

- lag(col, n) → trae el valor de la fila anterior (n pasos atrás).
- lead(col, n) → trae el valor de la fila siguiente (n pasos adelante).

2. Cálculos acumulativos

sum(), avg(), min(), max() dentro de una ventana ordenada.

3. Funciones de ranking

- row_number() → numera las filas en orden.
- rank() → ranking con posibles empates.
- dense_rank() → ranking consecutivo sin saltos.

In [0]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("location").orderBy("date")
df_covid = df_covid.withColumn("daily_cases", F.col("total_cases") - F.lag("total_cases").over(window_spec))
df_covid.filter(F.col("location") == "Colombia").select("date", "daily_cases").show(10)

# ✅ 9. Conclusiones
- Los **datos estructurados** (CSV, Parquet) son más fáciles de manejar pero requieren cuidado en el **esquema** y **particiones**.
- Los **datos semiestructurados** (JSON) necesitan transformación de estructuras anidadas.
- La **optimización** en Databricks combina técnicas de Spark (cache, repartition, broadcast) con almacenamiento eficiente (**Delta Lake**).